In [1]:
%config IPCompleter.greedy = True
%autosave 60

Autosaving every 60 seconds


## 1. NLP

### Cargar archivo

In [2]:
import pandas as pd
import os
ruta_archivo = os.path.join("imdb_dataset.csv")
df_criticas = pd.read_csv(ruta_archivo, encoding='iso-8859-2').sample(10000, replace=False)
#df_criticas = pd.read_csv(ruta_archivo, encoding='iso-8859-2')
df_criticas

,Review,Label
37295,Wow...sheer brilliance.<br /><br />Turning a t...,neg
49124,"Yesterday, I went alone to the cinema, because...",pos
13685,"A fantastic movie, and very overlooked. Gary h...",pos
49551,I tracked the trip two years ago on the intern...,pos
5675,"...un-funny and un-entertaining, possibly the ...",neg
...,...,...
12230,This movie was horrible and corny. James Agee ...,neg
13522,In & Out is a comedy with a simple premise. It...,pos
20166,Fires on The Plain (1959) ****<br /><br />You ...,pos
13596,What a great cast for this movie. The timing w...,pos


### Palabras de parada

In [3]:
import nltk
import string
# nltk.download('punkt')
# nltk.download('stopwords')
palabras_de_parada_ingles = set(nltk.corpus.stopwords.words('english') + list(string.punctuation) + ['...', '..'])
palabras_de_parada_ingles

{'!',
 '"',
 '#',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 '*',
 '+',
 ',',
 '-',
 '.',
 '..',
 '...',
 '/',
 ':',
 ';',
 '<',
 '=',
 '>',
 '?',
 '@',
 '[',
 '\\',
 ']',
 '^',
 '_',
 '`',
 'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 '

### Normalizar el texto

In [4]:
# LimpiadorTexto es una clase que nosotros creamos.
# Utilizamos la funcion normalizar aquí, en el notebook. y también en la API.
# Por tanto, para no tener código repetido, abstraímos la lógica a una clase externa.
from LimpiadorTexto import LimpiadorTexto
limpiador = LimpiadorTexto(palabras_de_parada_ingles)
df_criticas = df_criticas.apply(limpiador.normalizar_fila, axis=1)
df_criticas

,Review,Label
37295,wow sheer brilliance.turning thriller suspense...,neg
49124,yesterday went alone cinema mexico times movie...,pos
13685,fantastic movie overlooked gary never handsome...,pos
49551,tracked trip two years ago internet seen film ...,pos
5675,un-funny un-entertaining possibly worst movie ...,neg
...,...,...
12230,movie horrible corny james agee rolling grave....,neg
13522,comedy simple premise admirably succeeds missi...,pos
20166,fires plain 1959 see films like anymore fires ...,pos
13596,great cast movie timing excellent many clever ...,pos


### Obtener el vocabulario del problema

In [5]:
def obtener_vocabulario_problema(df_criticas_normalizadas):
    cuerpo_texto_normalizado = ' '.join(df_criticas_normalizadas['Review'].tolist())
    vocabulario_problema = cuerpo_texto_normalizado.split()
    vocabulario_ordenado = sorted(set(vocabulario_problema))
    return vocabulario_ordenado

def obtener_vocabulario_problema_y_posicion(vocabulario_del_problema_ordenado):
    vocabulario_y_posicion = {}
    for i, token in enumerate(vocabulario_del_problema_ordenado):
        vocabulario_y_posicion[token] = i
    return vocabulario_y_posicion

In [6]:
vocabulario_problema = obtener_vocabulario_problema_y_posicion(obtener_vocabulario_problema(df_criticas))
len(vocabulario_problema)

72975

### One Hot Encoding

In [7]:
import numpy as np

def obtener_one_hot_vector(critica, vocabulario_problema_y_posicion):
    one_hot_vector = np.zeros(len(vocabulario_problema_y_posicion), dtype=int)
    for token in critica.split():
        one_hot_vector[vocabulario_problema_y_posicion[token]] = 1
    return one_hot_vector

In [8]:
one_hots = []
indices = []
for indice, fila in df_criticas.iterrows():
    one_hot = obtener_one_hot_vector(fila['Review'], vocabulario_problema)
    indices.append(indice)
    one_hots.append(one_hot)

## 2. Crear y entrenar el modelo predictivo

In [ ]:
from sklearn.model_selection import train_test_split
# se cambia el nombre de la variable 'one_hots' para mas claridad
X = one_hots
Y = df_criticas['Label'].ravel()
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

X_train =  np.array(X_train)
X_test =  np.array(X_test)
Y_train = np.array(Y_train)
Y_test = np.array(Y_test)

### Liberar la memoria RAM

In [ ]:
import gc
lst = [df_criticas]
del df_criticas, one_hots, X, Y
del lst
gc.collect()

## Entrenar diferentes modelos con diferentes parámetros para encontrar el que produzca mejores resultados

Se entrenarán diferentes modelos con diferentes parámetros. Se guardará las métricas de cada modelo en un archivo para que luego puedan ser comparadas, y así determinar el mejor clasificador.

In [12]:
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
import json

def guardar_objeto_json(nombre_objeto, objeto):
    ruta_archivo = os.path.join(nombre_objeto)
    with open(ruta_archivo, 'w') as fp:
        json.dump(objeto, fp)
    
def obtener_metricas(clasificador):
    predicciones = clasificador.predict(X_test)
    metricas = {
        'accuracy': accuracy_score(Y_test, predicciones),
        'precision': precision_score(Y_test, predicciones, pos_label='pos'),
        'recall': recall_score(Y_test, predicciones, pos_label='pos'),
        'f1': f1_score(Y_test, predicciones, pos_label='pos')
    }
    return metricas

### Regresión logística con los parámetros:

- random_state=42
- solver='liblinear'

In [14]:
from sklearn.linear_model import LogisticRegression

clasificador = LogisticRegression(random_state=42, solver='liblinear')
clasificador.fit(X_train, Y_train)
metricas = obtener_metricas(clasificador)
metricas['model'] = 'LogisticRegression'
metricas['solver'] = 'liblinear'
metricas['random_state'] = 42
guardar_objeto_json('metricas-reg-log3.txt', metricas)

### Regresión logística con los parámetros:

- solver='lbfgs'
- n_jobs=-1

In [14]:
clasificador = LogisticRegression(solver='lbfgs', n_jobs=-1)
clasificador.fit(X_train, Y_train)
metricas = obtener_metricas(clasificador)
metricas['model'] = 'LogisticRegression'
metricas['solver'] = 'lbfgs'
guardar_objeto_json('metricas-reg-log4.txt', metricas)

### Regresión logística con los parámetros:

- solver='newton-cg'
- n_jobs=-1

In [16]:
clasificador = LogisticRegression(solver='newton-cg', n_jobs=-1)
clasificador.fit(X_train, Y_train)
metricas = obtener_metricas(clasificador)
metricas['model'] = 'LogisticRegression'
metricas['solver'] = 'newton-cg'
guardar_objeto_json('metricas-reg-log5.txt', metricas)

### Gradient Boosted Trees con los parámetros:

- n_estimators = 100
- max_depth = 10

In [13]:
from hyperopt import fmin, tpe, hp, STATUS_OK,Trials
from xgboost.sklearn import XGBClassifier

clasificador = XGBClassifier(n_estimators=100, max_depth=10 )
clasificador.fit(X_train, Y_train)
metricas = obtener_metricas(clasificador)
metricas['model'] = 'GradientBoostedTrees'
metricas['n_estimators'] = 100
metricas['max_depth'] = 10
guardar_objeto_json('metricas-GradientBoostedTrees.txt', metricas)

### Random Forest con los parámetros:

- criterion = gini
- max_depth = 18
- max_features = 146
- n_estimators = 373

In [20]:
from sklearn.ensemble import RandomForestClassifier

clasificador = RandomForestClassifier(criterion='gini', max_depth=18, max_features=146, n_estimators=373)
clasificador.fit(X_train, Y_train)
metricas = obtener_metricas(clasificador)
metricas['model'] = 'RandomForest'
metricas['n_estimators'] = 300
metricas['max_depth'] = 15
metricas['criterion'] = 'entropy'
guardar_objeto_json('metricas-RandomForest.txt', metricas)

## Guardar el clasificador como un archivo

In [14]:
import pickle
ruta_archivo_clasificador = os.path.join('clasificador-regresion-logistica.pkl')
archivo_clasificador = open(ruta_archivo_clasificador, 'wb')
pickle.dump(clasificador_reg_log, archivo_clasificador)
archivo_clasificador.close()

### Guardar el vocabulario del problema como un archivo

In [15]:
ruta_archivo_vocabulario = os.path.join('vocabulario-problema.pkl')
archivo_vacabulario = open(ruta_archivo_vocabulario, 'wb')
pickle.dump(vocabulario_problema, archivo_vacabulario)
archivo_vacabulario.close()

### Guardar las palabras de parada como un archivo

In [16]:
ruta_archivo_palabras_parada = os.path.join('palabras-parada.pkl')
archivo_palabras_parada = open(ruta_archivo_palabras_parada, 'wb')
pickle.dump(palabras_de_parada_ingles, archivo_palabras_parada)
archivo_palabras_parada.close()